# table family_cancer_fgender_degree.csv (use sql2 or sql3, each works)

In [ ]:
# sql 1
# check if family_antecedents_treatment_line.EHR is subset of family_antecedents.EHR

select (count(t1.EHR) = count(t2.EHR)) from 
(
    select DISTINCT `family_antecedents_treatment_line`.ehr as EHR from `family_antecedents_treatment_line` 
    union 
    SELECT DISTINCT `family_antecedents`.EHR as EHR from `family_antecedents`) t2, 
(select DISTINCT `family_antecedents`.EHR as EHR from `family_antecedents`) t1

In [ ]:
# sql 2
select DISTINCT
tt2.EHR as Patient_id,
case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.Family
end as Family,
case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.FamilyCancer
end as FamilyCancer,
case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 and tt1.FamilyGenderDegree LIKE '%Female%' then "Female"
when tt2.has_family_cancer_antecedents = 1 and tt1.FamilyGenderDegree LIKE '%Male%' then "Male"
end as FamilyGender,
case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.FamilyDegree
end as FamilyDegree

from `family_antecedents` tt2 

LEFT OUTER JOIN

(   SELECT distinct t1.ehr as EHR, t2.cancer_type_translated as FamilyCancer, t3.Gender_Degree as FamilyGenderDegree, t3.Degree as FamilyDegree, t3.familyMember_Translated as Family FROM 
    `family_antecedents_treatment_line` t1 
    
    LEFT JOIN 
    `familyRelationshipsENG` t3 ON t1.family_member = t3.familyMember 
    
    LEFT JOIN 
    `family_antecedents_treatment_line_translated` t2 ON t1.cancer_type=t2.cancer_type
) tt1 ON tt1.EHR = tt2.EHR


In [ ]:
# sql 3

SELECT DISTINCT
X.EHR as Patient_id,

case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.Family
end as Family,

case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.FamilyCancer
end as FamilyCancer,

case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 and tt1.FamilyGenderDegree LIKE '%Female%' then "Female"
when tt2.has_family_cancer_antecedents = 1 and tt1.FamilyGenderDegree LIKE '%Male%' then "Male"
end as FamilyGender,

case 
when tt2.has_family_cancer_antecedents = -1 then NULL
when tt2.has_family_cancer_antecedents = 0 then "No"
when tt2.has_family_cancer_antecedents = 1 then tt1.FamilyDegree
end as FamilyDegree from 

(   select DISTINCT `family_antecedents`.EHR as EHR from `family_antecedents` 
    
    union 
    select DISTINCT ehr as EHR from `family_antecedents_treatment_line`
) X 

LEFT OUTER JOIN
`family_antecedents` tt2 on tt2.EHR = X.EHR

LEFT OUTER JOIN 
(   SELECT distinct t1.ehr as EHR, t2.cancer_type_translated as FamilyCancer, t3.Gender_Degree as FamilyGenderDegree, t3.Degree as FamilyDegree, t3.familyMember_Translated as Family FROM 
    `family_antecedents_treatment_line` t1 
    LEFT JOIN `familyRelationshipsENG` t3 ON t1.family_member = t3.familyMember 
    LEFT JOIN `family_antecedents_treatment_line_translated` t2 ON t1.cancer_type=t2.cancer_type
) tt1 ON tt1.EHR = X.EHR

# table mutation.csv (use sql 4)

In [ ]:
# sql 4

select EHR as Patient_id, Biomarker from

# positive biomarker table
(   SELECT DISTINCT `EHR`, `study_performed` as Biomarker FROM `biomarker_mutation` WHERE (`study_performed` = 'MET' and `result`!='3.0') or (`study_performed` = 'KRAS' and `result`!='0.0') or (`study_performed` = 'BRAF' and `result`!='0.0') or (`study_performed` = 'HER2Mut' and `result`!='0.0') or (`study_performed` = 'RET' and `result`!='0.0') or (`study_performed` = 'FGFR1' and `result`!='0.0') or (`study_performed` = 'NTKR' and `result`!='0.0') 
    UNION

    SELECT DISTINCT `EHR`, "ALK" as Biomarker FROM `alk_mutation` WHERE `result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0'
    UNION 

    SELECT distinct `EHR`, "EGFR" as Biomarker FROM `egfr_mutation` WHERE `result_egfr_t790m`=1.0 or `result_egfr_t790`=1.0 or `result_egfr_exon19`=1.0 or `result_egfr_exon21`=1.0 or `result_egfr_nos`=1.0 or `result_egfr_others`=1.0 or `result_egfr_exon20`=1.0 or `specify_result_egfr_exon20`=1.0 or `specify_result_egfr_others`=1.0
    UNION

    SELECT DISTINCT `EHR`, "HER2" as Biomarker FROM `her2_mutation` WHERE (`result_her2_ihq` = '1.0' or `result_her2_fish` = '1.0')
    UNION

    SELECT DISTINCT `EHR`, "ROS1" as Biomarker FROM `ros1_mutation` WHERE (`result_ros1_ihq` = '1.0' or `result_ros1_fish` = '1.0')
    UNION 

    SELECT DISTINCT `EHR`, "PDL1" as Biomarker FROM `pdl1_expression` WHERE `result` = '1.0'
) as postive

UNION    

# negative biomarker table
SELECT DISTINCT negative.EHR, "NoMutation" as Biomarker FROM 
(   SELECT DISTINCT `EHR` FROM `biomarker_mutation` WHERE (study_performed='MET' and result = 3.0) or (study_performed='KRAS' and result=0.0) or (study_performed= 'BRAF' and result=0.0) or (study_performed='HER2Mut' and result=0.0) or (study_performed='RET' and result=0.0) or (study_performed='FGFR1' and result=0.0) or (study_performed='NTKR' and result=0.0)
    UNION

    SELECT DISTINCT t1.EHR FROM `alk_mutation` as t1 WHERE t1.EHR NOT IN (SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') AND t1.EHR NOT IN (SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` is NULL AND `result_alk_fish` is NULL AND `result_alk_rna` is NULL) 
    UNION

    SELECT DISTINCT `EHR` FROM `egfr_mutation` WHERE `result_egfr_negative` = '1.0'
    UNION

    SELECT DISTINCT t2.EHR FROM `her2_mutation` as t2 WHERE t2.EHR NOT IN (SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE `result_her2_ihq` = '1.0' or `result_her2_fish` = '1.0') and t2.EHR NOT IN (SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE `result_her2_ihq` IS NULL and `result_her2_fish` IS NULL)
    UNION

    SELECT DISTINCT t3.EHR FROM `ros1_mutation` as t3 WHERE t3.EHR NOT IN (SELECT DISTINCT EHR FROM `ros1_mutation` WHERE `result_ros1_ihq` = '1.0' or `result_ros1_fish` = '1.0') and t3.EHR NOT IN (SELECT DISTINCT EHR FROM `ros1_mutation` WHERE `result_ros1_ihq` IS NULL and `result_ros1_fish` IS NULL)
    UNION

    SELECT DISTINCT `EHR` FROM `pdl1_expression` WHERE `result` = '0.0'

) as negative WHERE negative.EHR NOT IN (
    SELECT DISTINCT `EHR` FROM `biomarker_mutation` WHERE (`study_performed` = 'MET' and `result`!='3.0') or (`study_performed` = 'KRAS' and `result`!='0.0') or (`study_performed` = 'BRAF' and `result`!='0.0') or (`study_performed` = 'HER2Mut' and `result`!='0.0') or (`study_performed` = 'RET' and `result`!='0.0') or (`study_performed` = 'FGFR1' and `result`!='0.0') or (`study_performed` = 'NTKR' and `result`!='0.0') 
    UNION

    SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0'
    UNION 

    SELECT distinct `EHR` FROM `egfr_mutation` WHERE `result_egfr_t790m`=1.0 or `result_egfr_t790`=1.0 or `result_egfr_exon19`=1.0 or `result_egfr_exon21`=1.0 or `result_egfr_nos`=1.0 or `result_egfr_others`=1.0 or `result_egfr_exon20`=1.0 or `specify_result_egfr_exon20`=1.0 or `specify_result_egfr_others`=1.0
    UNION

    SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE (`result_her2_ihq` = '1.0' or `result_her2_fish` = '1.0')
    UNION

    SELECT DISTINCT `EHR` FROM `ros1_mutation` WHERE (`result_ros1_ihq` = '1.0' or `result_ros1_fish` = '1.0')
    UNION 

    SELECT DISTINCT `EHR` FROM `pdl1_expression` WHERE `result` = '1.0'
)

UNION     

# null biomarker table
SELECT DISTINCT null_mutation.EHR, NULL as Biomarker FROM 
(
    SELECT DISTINCT `EHR` FROM `biomarker_mutation` 
    UNION

    SELECT DISTINCT EHR from `alk_mutation` # where result_alk_ihq is NULL and result_alk_fish is NULL and result_alk_rna is NULL
    UNION

    SELECT DISTINCT EHR from `egfr_mutation` 
    UNION

    SELECT DISTINCT EHR from `her2_mutation` # where result_her2_ihq is NULL and result_her2_fish is NULL
    UNION

    SELECT DISTINCT EHR from `ros1_mutation` # where result_ros1_ihq is NULL and  result_ros1_fish is NULL
    UNION 

    SELECT DISTINCT EHR from `pdl1_expression` # where result != '1.0' and result != '0.0'
) as null_mutation where null_mutation.EHR not in ( # exclude positive

    SELECT DISTINCT `EHR` FROM `biomarker_mutation` WHERE (`study_performed` = 'MET' and `result`!='3.0') or (`study_performed` = 'KRAS' and `result`!='0.0') or (`study_performed` = 'BRAF' and `result`!='0.0') or (`study_performed` = 'HER2Mut' and `result`!='0.0') or (`study_performed` = 'RET' and `result`!='0.0') or (`study_performed` = 'FGFR1' and `result`!='0.0') or (`study_performed` = 'NTKR' and `result`!='0.0') 
    UNION

    SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0'
    UNION 

    SELECT distinct `EHR` FROM `egfr_mutation` WHERE `result_egfr_t790m`=1.0 or `result_egfr_t790`=1.0 or `result_egfr_exon19`=1.0 or `result_egfr_exon21`=1.0 or `result_egfr_nos`=1.0 or `result_egfr_others`=1.0 or `result_egfr_exon20`=1.0 or `specify_result_egfr_exon20`=1.0 or `specify_result_egfr_others`=1.0
    UNION

    SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE (`result_her2_ihq` = '1.0' or `result_her2_fish` = '1.0')
    UNION

    SELECT DISTINCT `EHR` FROM `ros1_mutation` WHERE (`result_ros1_ihq` = '1.0' or `result_ros1_fish` = '1.0')
    UNION 

    SELECT DISTINCT `EHR` FROM `pdl1_expression` WHERE `result` = '1.0'
    UNION # exclude negative

    SELECT DISTINCT `EHR` FROM `biomarker_mutation` WHERE (study_performed='MET' and result = 3.0) or (study_performed='KRAS' and result=0.0) or (study_performed= 'BRAF' and result=0.0) or (study_performed='HER2Mut' and result=0.0) or (study_performed='RET' and result=0.0) or (study_performed='FGFR1' and result=0.0) or (study_performed='NTKR' and result=0.0)
    UNION

    SELECT DISTINCT t1.EHR FROM `alk_mutation` as t1 WHERE t1.EHR NOT IN (SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') AND t1.EHR NOT IN (SELECT DISTINCT `EHR` FROM `alk_mutation` WHERE `result_alk_ihq` is NULL AND `result_alk_fish` is NULL AND `result_alk_rna` is NULL) 
    UNION

    SELECT DISTINCT `EHR` FROM `egfr_mutation` WHERE `result_egfr_negative` = '1.0'
    UNION

    SELECT DISTINCT t2.EHR FROM `her2_mutation` as t2 WHERE t2.EHR NOT IN (SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE `result_her2_ihq` = '1.0' or `result_her2_fish` = '1.0') and t2.EHR NOT IN (SELECT DISTINCT `EHR` FROM `her2_mutation` WHERE `result_her2_ihq` IS NULL and `result_her2_fish` IS NULL)
    UNION

    SELECT DISTINCT t3.EHR FROM `ros1_mutation` as t3 WHERE t3.EHR NOT IN (SELECT DISTINCT EHR FROM `ros1_mutation` WHERE `result_ros1_ihq` = '1.0' or `result_ros1_fish` = '1.0') and t3.EHR NOT IN (SELECT DISTINCT EHR FROM `ros1_mutation` WHERE `result_ros1_ihq` IS NULL and `result_ros1_fish` IS NULL)
    UNION

    SELECT DISTINCT `EHR` FROM `pdl1_expression` WHERE `result` = '0.0'
)

# table gender_age_smoker.csv  (use sql 5)

In [ ]:
# sql 5

select X.Patient_id, tt1.Age, tt2.Gender, tt3.Smoker
from 
(
    (select DISTINCT EHRNew as Patient_id FROM (SELECT distinct t1.EHR as EHRNew, CEIL(DATEDIFF(t1.date,t2.birth_date)/365) as age FROM diagnosis t1, patient t2 where t1.EHR=t2.EHR) t3 WHERE t3.age is NOT NULL and t3.age>0)
    UNION
    (SELECT DISTINCT t1.EHR as Patient_id FROM `patient` t1, `patient_translated` t2 WHERE t1.gender=t2.gender)
    UNION 
    (SELECT distinct EHR as Patient_id from smoker_information)
) X

LEFT OUTER JOIN 
(SELECT distinct EHRNew as Patient_id, Age FROM (SELECT distinct t1.EHR as EHRNew, CEIL(DATEDIFF(t1.date,t2.birth_date)/365) as age FROM diagnosis t1, patient t2 where t1.EHR=t2.EHR) t3 WHERE t3.age is NOT NULL and t3.age>0) tt1 on X.Patient_id = tt1.Patient_id

LEFT OUTER JOIN
(SELECT DISTINCT t1.EHR as Patient_id, t2.gender_translated as Gender FROM `patient` t1, `patient_translated` t2 WHERE t1.gender=t2.gender) tt2 ON X.Patient_id = tt2.Patient_id

LEFT OUTER JOIN
(SELECT distinct EHR as Patient_id, 
case 
when smoking_habit in (-1,3) then NULL
when smoking_habit = 0 then 'Never-Smoker'
when smoking_habit = 1 then 'Former-Smoker'
when smoking_habit = 2 then 'Current-Smoker'
end as Smoker from smoker_information) tt3 ON X.Patient_id = tt3.Patient_id